In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
  0.279752 
  0.341751 
 -0.869452 
  0.221356 
  0.0925629
  0.385457 
  0.224419 

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 3.7533655654172486 rad about [-0.016907547855919466,0.2173819525446583,-0.9759401731322325], translation: Vec(0.7202320682909007,0.3538792910032498,0.3276280127708483)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(-0.35933380333107917,1.9178204805394363,0.6534809126528677)

In [12]:
mass(mechanism)

0.9431674736504114

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(0.037915035757811744,1.4589610697611022,0.9043336596874957)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.034130556579783033,0.10570053550349046,-0.0873033887001578), linear: Vec(-0.2229607974745511,0.03417552224331094,-0.04578751375594736)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.166964   0.504557   0.60875   …  -0.0       -0.0       -0.0      
 -0.517079  -0.468627  -0.402481     -0.0       -0.0       -0.0      
  0.427081   0.725128  -0.68369      -0.0       -0.0       -0.0      
  1.09071    0.704884  -0.275223      0.504557   0.60875   -0.612246 
 -0.167184  -0.220738   0.708252     -0.468627  -0.402481  -0.786382 
  0.223989  -0.633127  -0.661996  …   0.725128  -0.68369   -0.0822021

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(0.6167448034242595,-0.6285106394933232,-0.14677992309855492), linear: Vec(0.7851308537638,-0.3780815690039123,-0.850639098931806)

In [18]:
mass_matrix(x)

7x7 Array{Float64,2}:
  1.05608      -1.12173      -0.623718     …   1.16149      -0.0364986 
 -1.12173       1.45323      -0.695614        -0.88679      -0.125493  
 -0.623718     -0.695614      1.82004          2.60209e-17   0.0831013 
 -8.32667e-17   0.368828     -1.16149         -4.16334e-17  -0.033382  
 -0.368828      1.66533e-16   0.88679         -1.38778e-16   0.0263229 
  1.16149      -0.88679      -2.77556e-17  …   0.943167      0.043269  
 -0.0364986    -0.125493      0.0831013        0.043269      0.00514269

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -0.690243   -0.732076   2.91525    …   0.98861   -0.335991    0.163169 
  0.0824188  -0.611185  -0.0254677     -0.143487   0.405212   -0.0182823
 -1.73633     1.91319   -0.0220157      0.964717  -1.37396    -0.0527899
  0.935641   -0.729029   0.0462037     -0.574153   0.577451    0.0273103
  0.764927   -0.524513  -0.187389       0.379607   0.74169     0.0289768
 -0.156688   -0.340344   1.44852    …   0.644834   0.0775304   0.0457597

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(-0.964220503551722,-0.6906699102232732,1.352678428381408), linear: Vec(-0.6258239337305629,0.6521125475777928,-0.14529904348213904)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(0.20161352191929097,0.8112094483080884,-0.8646920833648908), linear: Vec(-1.9961134866757866,0.46011802831961324,-0.7838663367259984)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.07038582513454873,-0.04883423083205911,-0.08664167393316163), linear: Vec(-0.15671503053469849,-0.10047717808956447,0.0231627715889917)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

Dict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "joint2": Revolut… => [0.48046471812754654]
  Joint "joint1": Quatern… => [-1.6894368715667403,-4.486933184949033,15.530955…

In [24]:
relative_acceleration(x, body1, body2, v̇)

SpatialAcceleration of "after_joint1" w.r.t "after_joint2" in "world":
angular: Vec(0.0011524886990566668,-0.16557776443729383,0.26373532512245557), linear: Vec(0.6089877576845621,0.03115262946115678,0.06971653253252282)

In [25]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [26]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3
        Vertex: body4, Edge: joint4
        Vertex: body6, Edge: joint6
          Vertex: body7, Edge: joint7
          Vertex: body8, Edge: joint8
        Vertex: body11, Edge: joint11
      Vertex: body5, Edge: joint5
        Vertex: body9, Edge: joint9
      Vertex: body10, Edge: joint10